#  Data Connections Exercise

## Ask a Legal Research Assistant Bot about the US Constitution

Your function should do the following:

* Read the US_Constitution.txt file inside the some_data folder
* Split this into chunks (you choose the size)
* Write this to a ChromaDB Vector Store
* Use Context Compression to return the relevant portion of the document to the question

In [ ]:
!pip install langchain openai chromadb tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.3 MB/s eta 0:00:00


In [ ]:
# Build a sample vectorDB
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
import tiktoken

In [ ]:
f = open("/content/api_key.txt")
api_key = f.read()
model = ChatOpenAI(openai_api_key = api_key)

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = api_key

In [ ]:
# PART ONE:
# LOAD "some_data/US_Constitution in a Document object
loader = TextLoader("/content/US_Constitution.txt")
documents = loader.load()


In [ ]:
# PART TWO
# Split the document into chunks (you choose how and what size)
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size = 500)
docs = text_splitter.split_documents(documents)

In [ ]:
# PART THREE
# EMBED THE Documents (now in chunks) to a persisted ChromaDB
embedding_function = OpenAIEmbeddings()
db = Chroma.from_documents(docs, embedding_function,persist_directory='./US_Constitution')


In [ ]:
db.persist()

In [ ]:
def us_constitution_helper(question):
    '''
    Takes in a question about the US Constitution and returns the most relevant
    part of the constitution. Notice it may not directly answer the actual question!

    Follow the steps below to fill out this function:
    '''
    # PART FOUR
    # Use ChatOpenAI and ContextualCompressionRetriever to return the most
    # relevant part of the documents.
    llm = ChatOpenAI(temperature=0)
    #insert LLM to LLMChainExtractor
    compressor = LLMChainExtractor.from_llm(llm)
    #Contextual Compression
    compression_retriever = ContextualCompressionRetriever(base_compressor=compressor,
                                                           base_retriever=db.as_retriever())

    compressed_docs = compression_retriever.get_relevant_documents(question)

    result = compressed_docs[0].page_content

    print(result)

## Example Usage:

Notice how it doesn't return an entire Document of a large chunk size, but instead the "compressed" version!

In [ ]:
print(us_constitution_helper("What is the 13th Amendment?"))

13th Amendment
Section 1
Neither slavery nor involuntary servitude, except as a punishment for crime whereof the party shall have been duly convicted, shall exist within the United States, or any place subject to their jurisdiction.
None


In [ ]:
print(us_constitution_helper("What is the 1st Amendment?"))

First Amendment
Congress shall make no law respecting an establishment of religion, or prohibiting the free exercise thereof; or abridging the freedom of speech, or of the press; or the right of the people peaceably to assemble, and to petition the Government for a redress of grievances.
None
